In [8]:
import cv2
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_classification
from sklearn.tree import DecisionTreeClassifier

In [14]:
X, y = make_classification(n_samples=1000, n_features=4,
                            n_informative=2, n_redundant=0,
                            random_state=0, shuffle=False)
base_classifier=DecisionTreeClassifier(max_depth=2)
clf = AdaBoostClassifier(base_classifier, 2048, random_state=0)
print('X',y.shape)
clf.fit(X, y)
clf.predict([[0, 0, 0, 0]])
#clf.score(X, y)

X (1000,)


1

In [15]:
#ACF:
#1237 osób
#4280 tła - zupełnie bez palety

Viano
7124/6700 zdjęć
9047 palet
37 scen
181 obrazów per scena

Można wziąć 20 procent zróżnicowanych scen, czyli kilka i na nich nauczyć. 
Negatywnych wziąć losowo 2x więcej niż pozytywnych.

Można też zrobić rozkład wysokości punktów i obciąć od góry i dołu zdjęcia tak, 
żeby wiekszość się mieściła.

Kolory można zaznaczyć tak, że z zaznaczonej ramki wyciąć ształt palety i już. A tło wyciąć z tła.

stack_1 i stack_2 odrzucam, bo za dużo zaznaczania, za bardzo krzywe i trudniej będzię oznaczyć kolor